# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# use pandas function to read in csv from WeatherPy output data folder
cities_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,laguna,38.4210,-121.4238,48.06,39.0,1.0,1.34,US,1.629948e+09
1,vaini,-21.2000,-175.2000,44.27,64.0,20.0,6.69,TO,1.629948e+09
2,tessalit,20.1986,1.0114,47.52,51.0,9.0,4.29,ML,1.629948e+09
3,khatanga,71.9667,102.5000,33.98,73.0,100.0,5.91,RU,1.629948e+09
4,busselton,-33.6500,115.3333,40.66,76.0,77.0,10.72,AU,1.629948e+09
...,...,...,...,...,...,...,...,...,...
533,honiara,-9.4333,159.9500,48.58,67.0,70.0,4.21,SB,1.629948e+09
534,iralaya,15.0000,-83.2333,46.74,82.0,99.0,4.34,HN,1.629948e+09
535,constitucion,-35.3333,-72.4167,36.89,86.0,99.0,1.90,CL,1.629948e+09
536,cayenne,4.9333,-52.3333,47.15,81.0,0.0,0.45,GF,1.629948e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# use stored google api-key to 
gmaps.configure(api_key=g_key)

In [4]:
# pull latitude, longitude, and humidity into lists
lat = cities_df['Lat'].tolist()
lng = cities_df['Lng'].tolist()
#humidity = cities_df['Humidity'].tolist()

# create list of locations
indexval = np.arange(0, 538, 1)
locations = [(float(lat[ind]), float(lng[ind])) for ind in indexval]
humidity = [float(pct) for pct in cities_df['Humidity']]

In [8]:
# generate map figure, edit parameters to minimize map repeats
fig = gmaps.figure(layout={'width': '800px', 'height': '500px'}, center= (0,0), zoom_level=2)

# generate heatmap layer weighted by humidity percentage values for each city
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity)

# add the heatmap layer to the figure
fig.add_layer(heatmap_layer)

# display the map
fig

Figure(layout=FigureLayout(height='500px', width='800px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# define conditions for 'ideal' weather
hightemp = 50
lowtemp = 47
maxwind = 10
maxcloud = 0

# filter dataframe based on conditions
hotel_df = cities_df.loc[cities_df['Max Temp'] < hightemp]
hotel_df = hotel_df.loc[hotel_df['Max Temp'] > lowtemp]
hotel_df = hotel_df.loc[hotel_df['Wind Speed'] < maxwind]
hotel_df = hotel_df.loc[hotel_df['Cloudiness'] == maxcloud]

# display the reduced dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
180,karratha,-20.7377,116.8463,48.41,27.0,0.0,3.11,AU,1.629948e+09
214,port hedland,-20.3167,118.5667,49.24,12.0,0.0,4.12,AU,1.629948e+09
240,sangin,32.0728,64.8359,47.50,13.0,0.0,2.58,AF,1.629948e+09
280,qingdao,36.0986,120.3719,47.99,51.0,0.0,5.00,CN,1.629948e+09
314,annau,37.8876,58.5160,47.40,28.0,0.0,2.06,TM,1.629948e+09
322,minab,27.1467,57.0801,49.62,58.0,0.0,1.18,IR,1.629948e+09
386,baherden,38.4362,57.4316,49.86,18.0,0.0,1.10,TM,1.629948e+09
433,elat,29.5581,34.9482,48.06,51.0,0.0,6.17,IL,1.629948e+09
437,ibra,22.6906,58.5334,47.43,62.0,0.0,2.44,OM,1.629948e+09
451,conceicao do araguaia,-8.2578,-49.2647,47.04,65.0,0.0,0.51,BR,1.629948e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
# generate empty list for hotel names
hotelnames = []

# set url for nearby search
nearby_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# loop through cities fitting ideal weather conditions to find hotels
for index, row in hotel_df.iterrows():
    # establish search parameters
    target_coords = f"{row['Lat']}, {row['Lng']}"
    target_search = "Hotel"
    target_radius = 5000

    # generate parameter dictionary
    params = {
    "location": target_coords,
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key}
    
    # make api call
    response = requests.get(nearby_url, params).json()

    # pull out name of first hotel result
    try:
        hotelname = response["results"][0]["name"]
    except IndexError:
        hotelname = None
    
    # append hotel name to list
    hotelnames.append(hotelname)
    
# add hotel names to dataframe
hotel_df['Hotel Name'] = hotelnames

# drop cities with no hotel in range
hotel_df.dropna(axis=0, inplace=True)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
180,karratha,-20.7377,116.8463,48.41,27.0,0.0,3.11,AU,1.629948e+09,Karratha International Hotel
214,port hedland,-20.3167,118.5667,49.24,12.0,0.0,4.12,AU,1.629948e+09,Hospitality Port Hedland
280,qingdao,36.0986,120.3719,47.99,51.0,0.0,5.00,CN,1.629948e+09,Shangri-La Hotel Qingdao
322,minab,27.1467,57.0801,49.62,58.0,0.0,1.18,IR,1.629948e+09,Minab Tourism Hotel
433,elat,29.5581,34.9482,48.06,51.0,0.0,6.17,IL,1.629948e+09,Dan Eilat Hotel
437,ibra,22.6906,58.5334,47.43,62.0,0.0,2.44,OM,1.629948e+09,Ibra Hotel
451,conceicao do araguaia,-8.2578,-49.2647,47.04,65.0,0.0,0.51,BR,1.629948e+09,Tarumã Tropical Hotel
469,aswan,24.0934,32.9070,47.34,26.0,0.0,3.09,EG,1.629948e+09,Sofitel Legend Old Cataract Aswan
536,cayenne,4.9333,-52.3333,47.15,81.0,0.0,0.45,GF,1.629948e+09,Hotel Amazonia Cayenne


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='500px', width='800px'))